In [1]:
import requests
import geopandas as gpd
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union
import osm2geojson  # pip install osm2geojson

# Grenoble bounding box
north, south = 45.36962853982034, 45.04131660493893
west, east = 5.484539716781988, 5.9934400014814155

# Overpass query for building ways + relations
overpass_url = "https://overpass-api.de/api/interpreter"
query = f"""
[out:json][timeout:180];
(
  way["building"]({south},{west},{north},{east});
  relation["building"]({south},{west},{north},{east});
);
out body;
>;
out skel qt;
"""

# Send the Overpass request
print("⏳ Downloading OSM data for Grenoble...")
response = requests.get(overpass_url, params={"data": query})
data = response.json()

# Convert to GeoJSON (handles relations)
print("🔄 Converting to GeoJSON...")
geojson = osm2geojson.json2geojson(data)

# Load into GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson["features"])
gdf.set_crs("EPSG:4326", inplace=True)

# Clean invalid geometries
print("🧼 Cleaning geometries...")
gdf["geometry"] = gdf["geometry"].buffer(0)

# Merge all overlapping/touching buildings
print("🔗 Merging building footprints...")
merged = unary_union(gdf["geometry"])

# Create output GeoDataFrame
if isinstance(merged, MultiPolygon):
    merged_gdf = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=gdf.crs)
else:
    merged_gdf = gpd.GeoDataFrame(geometry=[merged], crs=gdf.crs)

# Save to file
output_file = "grenoble_buildings_merged.gpkg"
merged_gdf.to_file(output_file, driver="GPKG")

print(f"✅ Done! Merged {len(gdf)} buildings into {len(merged_gdf)} final blocks.")
print(f"📦 Output saved to: {output_file}")


⏳ Downloading OSM data for Grenoble...
🔄 Converting to GeoJSON...
🧼 Cleaning geometries...
🔗 Merging building footprints...


/Users/gabeiras3j/opt/anaconda3/envs/cleanproj/lib/python3.10/site-packages/shapely/set_operations.py:543: RuntimeWarning: invalid value encountered in create_collection
  collections = lib.create_collection(


✅ Done! Merged 240916 buildings into 140809 final blocks.
📦 Output saved to: grenoble_buildings_merged.gpkg
